In [13]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch_geometric.nn import GAE
from torch_geometric.utils import is_undirected, to_undirected


from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

from model.GCN_GRU import GCN_GRU, Decoder
from utils import *

In [2]:
shop_col = 'stonc_6_label'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

embedding_dim = 64
entity_dim = 8
epochs = 1000
learning_rate = 0.0001
batch_size = 1000

weights_path = 'GCN_GRU_Link_Prediction'

In [3]:
data_path = './data'

chid_dict_file = 'sample/sample_50k_idx_map.npy'
cdtx_file = 'sample/sample_50k_cdtx.csv'
cust_file = 'preprocessed/df_cust_log_without_shop.csv'

chid_path = os.path.join(data_path, chid_dict_file)
cdtx_path = os.path.join(data_path, cdtx_file)
cust_path = os.path.join(data_path, cust_file)

In [4]:
df_cdtx, df_cust, n_users, n_shops = read_sample_files(cdtx_path,
                                                       cust_path,
                                                       chid_path,
                                                       shop_col)

Start reading cdtx file...
Finish reading cdtx file !
Start reading cust file...
Finish reading cust file !
Start maping encodeing...
Finish !!


In [5]:
list_months = sorted(df_cdtx.csmdt.unique())
ma = df_cdtx.groupby(['chid', 'csmdt']).objam.sum().max()

ignore_cols = ['chid', 'data_dt']
category_cols = [f'category_{i+1}' for i in range(6)]
numeric_cols = list(set(df_cust.columns) - set(category_cols) - set(ignore_cols))

emb_dims = list(df_cust[category_cols].nunique())

input_dim = len(category_cols)*entity_dim + len(numeric_cols)

In [6]:
edge_dict = {}
edge_weights_dict = {}
for month in tqdm(list_months):
    edges = df_cdtx[df_cdtx.csmdt==month].groupby(['chid', shop_col]).objam.sum()
    edge_pairs = np.stack([np.array(i) for i in edges.index]).T
    edge_pairs = torch.LongTensor(edge_pairs)
    
    edge_weights = np.log(edges.values+1)
    edge_weights = torch.Tensor(edge_weights)
    
    if not is_undirected(edge_pairs):
        edge_pairs = torch.cat([edge_pairs, edge_pairs[[1,0],:]], -1)
        edge_weights = edge_weights.repeat(2)
    
    edge_dict[month] = edge_pairs
    edge_weights_dict[month] = edge_weights

100%|██████████| 25/25 [00:19<00:00,  1.30it/s]


In [9]:
model = GAE(GCN_GRU(input_dim, embedding_dim, entity_dim, emb_dims), Decoder(embedding_dim, 6)).to(device)

x_num = []
x_cat = []
y = []
for i in list_months:
    cust_num_features = df_cust[df_cust.data_dt==i][numeric_cols].to_numpy()
    cust_num_features = torch.Tensor(cust_num_features)
    cust_cat_features = df_cust[df_cust.data_dt==i][category_cols].to_numpy()
    cust_cat_features = torch.LongTensor(cust_cat_features)
    
    shop_num_features = torch.zeros(n_shops, cust_num_features.shape[1])
    shop_cat_features = torch.zeros(n_shops, cust_cat_features.shape[1]).long()
    
    temp_y = df_cust[df_cust.data_dt==i]['objam'].to_numpy()*ma
    ind1 = (temp_y >0) & (temp_y <= 1e4)
    ind2 = (temp_y >1e4) & (temp_y <= 5e4)
    ind3 = (temp_y >5e4) & (temp_y <= 1e5)
    ind4 = (temp_y >1e5) & (temp_y <= 3e5)
    ind5 = (temp_y >3e5)

    temp_y[ind1] = 1
    temp_y[ind2] = 2
    temp_y[ind3] = 3
    temp_y[ind4] = 4
    temp_y[ind5] = 5
    
    x_num.append(torch.cat([cust_num_features, shop_num_features], 0).to(device))
    x_cat.append(torch.cat([cust_cat_features, shop_cat_features], 0).to(device))
    y.append(torch.LongTensor(temp_y).to(device))
    
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
pos_edge_index = [i.to(device) for i in edge_dict.values()]
pos_edge_weigts = [i.to(device) for i in edge_weights_dict.values()]

In [10]:
model.encoder.load_state_dict(torch.load(weights_path))
model.eval()

criterion = torch.nn.CrossEntropyLoss()

GAE(
  (encoder): GCN_GRU(
    (emb_layers): ModuleList(
      (0): Embedding(4, 8)
      (1): Embedding(7, 8)
      (2): Embedding(3, 8)
      (3): Embedding(28, 8)
      (4): Embedding(10, 8)
      (5): Embedding(31, 8)
    )
    (gcn1_dict): ModuleDict(
      (0): GCNConv(72, 128)
      (1): GCNConv(72, 128)
      (2): GCNConv(72, 128)
      (3): GCNConv(72, 128)
      (4): GCNConv(72, 128)
      (5): GCNConv(72, 128)
      (6): GCNConv(72, 128)
      (7): GCNConv(72, 128)
      (8): GCNConv(72, 128)
      (9): GCNConv(72, 128)
      (10): GCNConv(72, 128)
      (11): GCNConv(72, 128)
    )
    (gcn2_dict): ModuleDict(
      (0): GCNConv(128, 64)
      (1): GCNConv(128, 64)
      (2): GCNConv(128, 64)
      (3): GCNConv(128, 64)
      (4): GCNConv(128, 64)
      (5): GCNConv(128, 64)
      (6): GCNConv(128, 64)
      (7): GCNConv(128, 64)
      (8): GCNConv(128, 64)
      (9): GCNConv(128, 64)
      (10): GCNConv(128, 64)
      (11): GCNConv(128, 64)
    )
    (rnn): GRU(64, 64)
  )

In [14]:
def train():
    model.train()
    train_output = []
    train_y = np.array([])
    for i in tqdm(range(10)):
        
        train_dataset = TensorDataset(y[i+12])
        train_loader = DataLoader(dataset=train_dataset, shuffle=False, batch_size=batch_size)
        
        for j, true_y in enumerate(train_loader):
            optimizer.zero_grad()
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[j*batch_size:(j+1)*batch_size]).sigmoid()
            loss = criterion(output, true_y[0])
            train_output.append(output.cpu().detach().numpy())
            train_y = np.concatenate([train_y, true_y[0].cpu().detach().numpy().reshape(-1)])
        
            loss.backward(retain_graph=True)
            optimizer.step()
        
            
    return loss/10, train_output, train_y

In [15]:
def test():
    model.eval()
    test_output = []
    test_y = np.array([])
    
    for i in range(10,12):

        with torch.no_grad():
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[:n_users]).sigmoid()
            test_output.append(output.cpu().detach().numpy())
            test_y = np.concatenate([test_y, y[i+12].cpu().detach().numpy().reshape(-1)])
            
    return test_output, test_y

In [16]:
for epoch in range(epochs):
    loss, train_output, train_y  = train()
    
    test_output, test_y = test()
    
    print('train',f1_score(train_y, np.argmax(np.concatenate(train_output,0),-1), average='macro'))
    print('test', f1_score(test_y, np.argmax(np.concatenate(test_output,0),-1), average='macro'))

  0%|          | 0/10 [00:00<?, ?it/s]

epoch:1
train AUC:0.7883245518316017,test AUC:0.9102383723396988
epoch:1
train F1:0.8405781281269573,test F1:0.8911498528676919


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:2
train AUC:0.8997088843234108,test AUC:0.913773714899365
epoch:2
train F1:0.8861789643509427,test F1:0.8939938488268634


  0%|          | 0/10 [00:00<?, ?it/s]

...

epoch:80
train AUC:0.9055478752298084,test AUC:0.9116988739751016
epoch:80
train F1:0.8972296515958768,test F1:0.8934933742762644


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:81
train AUC:0.9057494069782741,test AUC:0.911829578678518
epoch:81
train F1:0.897476206421003,test F1:0.8934482188670024


 70%|███████   | 7/10 [01:36<00:41, 13.83s/it]


KeyboardInterrupt: 